In [14]:
from IPython import display
import torch
from dotenv import load_dotenv
import shutil
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import roboflow
import ultralytics
from ultralytics import YOLO

In [8]:
display.clear_output()
ultralytics.checks()

Ultralytics 8.3.104  Python-3.8.10 torch-2.4.1+cpu CPU (AMD Ryzen 5 3600 6-Core Processor)
Setup complete  (12 CPUs, 15.9 GB RAM, 100.2/931.5 GB disk)


In [9]:
# Em apple sillicon, verificar se o MPS está disponível
print(torch.backends.mps.is_available())

False


In [10]:
# Carregar variáveis do ficheiro .env
load_dotenv(override=True)

# Obter variáveis
api_key = os.getenv("ROBOFLOW_API_KEY")
workspace = os.getenv("ROBOFLOW_WORKSPACE")
project_name = os.getenv("ROBOFLOW_PROJECT")
version = int(os.getenv("ROBOFLOW_VERSION", "1"))

In [20]:
# descarregar o dataset do roboflow, depois de etiquetadas as imagens e criado o dataset em Mac tive problemas de permissões e foi necessário dar permissões à pasta de conf. do roboflow:
# - sudo mkdir /Users/davidecarneiro/.config/roboflow (criar pasta onde vai guardar a conf.)
# - sudo chown -R davidecarneiro:staff ~/.config/roboflow (dar permissões ao meu user)
roboflow.login()

rf = roboflow.Roboflow(api_key=api_key) # Faz login no Roboflow com a chave lida

# substituir nome do workspace e do projeto
project = rf.workspace(workspace).project(project_name)
# se versão do dataset > 1, substituir pela versão correspondente
dataset = project.version(version).download("yolov8")

# Move o dataset para a pasta desejada
shutil.move(dataset.location, "./datasets/candy-crush-saga-v3-1")

# Atualiza o caminho no objeto
dataset.location = "./datasets/candy-crush-saga-v3-1"

# WARN: necessário verificar os paths no ficheiro data.yaml, após este ser descarregado
# alterar para:
# train: ./datasets/candy-crush-saga-v3-1/train/images
# val: ./datasets/candy-crush-saga-v3-1/valid/images
# test: ./datasets/candy-crush-saga-v3-1/test/images

SyntaxError: invalid syntax (3357263069.py, line 21)

In [26]:
# treinar o modelo
# lista de modelos pre-treinados disponível em https://docs.ultralytics.com/models/yolov8/#performance-metrics
model = YOLO("YOLOv8m.pt")  # carregar o modelo pre-treinado que se descarregou

# Treinar o modelo
results = model.train(data='./datasets/candy-crush-saga-v3-1/data.yaml', epochs=100, imgsz=640, device='cpu')  # windows amd cpu and gpu
# results = model.train(data='candy-crush-saga-v3-1', epochs=100, imgsz=640, device=[0, 1]) # intel/windows
# results = model.train(data='FoE-bot-2/data.yaml', epochs=100, imgsz=640, device='mps') # apple sillicon

New https://pypi.org/project/ultralytics/8.3.105 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.104  Python-3.8.10 torch-2.4.1+cpu CPU (AMD Ryzen 5 3600 6-Core Processor)
engine\trainer: task=detect, mode=train, model=YOLOv8m.pt, data=./datasets/candy-crush-saga-v3-1/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None

train: Scanning D:\githubProjects\2025-ia-trabalhopratico\datasets\candy-crush-saga-v3-1\train\labels... 21 images, 0 backgrounds, 0 corrupt: 100%|██████████| 21/21 [00:00<00:00, 477.27it/s]

train: New cache created: D:\githubProjects\2025-ia-trabalhopratico\datasets\candy-crush-saga-v3-1\train\labels.cache



val: Scanning D:\githubProjects\2025-ia-trabalhopratico\datasets\candy-crush-saga-v3-1\valid\labels... 6 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6/6 [00:00<00:00, 520.36it/s]

val: New cache created: D:\githubProjects\2025-ia-trabalhopratico\datasets\candy-crush-saga-v3-1\valid\labels.cache


...

    100/100         0G     0.4235     0.2192     0.8734        217        640: 100%|██████████| 2/2 [00:22<00:00, 11.05s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.31s/it]

                   all          6        303      0.994          1      0.995      0.912



100 epochs completed in 0.756 hours.
Optimizer stripped from runs\detect\train5\weights\last.pt, 52.0MB
Optimizer stripped from runs\detect\train5\weights\best.pt, 52.0MB

Validating runs\detect\train5\weights\best.pt...
Ultralytics 8.3.104  Python-3.8.10 torch-2.4.1+cpu CPU (AMD Ryzen 5 3600 6-Core Processor)
Model summary (fused): 92 layers, 25,843,234 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.92s/it]


                   all          6        303      0.993          1      0.995      0.913
                  blue          6         58      0.999          1      0.995      0.909
            color-bomb          3          3      0.962          1      0.995      0.995
                 green          6         55      0.999          1      0.995       0.88
                purple          6         55      0.998          1      0.995      0.908
                   red          6         66      0.999          1      0.995      0.892
                yellow          6         66      0.999          1      0.995      0.894
Speed: 1.5ms preprocess, 306.9ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs\detect\train5
